# Basic Chain

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"))

In [2]:
# ask the LLM a question
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks to streamline the testing process. This can include features such as test case generation, test execution, and results reporting. Langsmith can also provide integration with continuous integration and continuous deployment pipelines to ensure that tests are automatically run and monitored throughout the development process. Additionally, Langsmith can offer support for various types of testing, including unit testing, integration testing, and performance testing, to help ensure the overall quality of the software being developed.')

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# create a prompt template with {input} as the user input
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

# you can easily replace placeholder with the user input
print(prompt.format(input="how can langsmith help with testing?"))

# create a simple output parser to parse AIMessge to string
output_parser = StrOutputParser()

# a simple LLM chain using LCEL (langchain expression language)
chain = prompt | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})
type(response), response

System: You are world class technical documentation writer.
Human: how can langsmith help with testing?


(str,
 'Langsmith can help with testing in a number of ways:\n\n1. Automated testing: Langsmith can be used to write test scripts in various programming languages to automate the testing process. This can help speed up the testing process and ensure that the software is thoroughly tested.\n\n2. Test data generation: Langsmith can be used to generate test data for different scenarios, allowing testers to simulate real-world conditions and edge cases. This can help ensure that the software is robust and can handle a variety of inputs.\n\n3. Integration testing: Langsmith can be used to write scripts for integration testing, where different components of the software are tested together to ensure that they work seamlessly. This can help identify any issues with how the components interact with each other.\n\n4. Performance testing: Langsmith can be used to write scripts for performance testing, where the software is tested under different load conditions to ensure that it can handle the e

# Retrieval Chain

Load documents from web and ingest documents into a vectorstore. Here we use OpenAI's embedding model by default and a light weight local vectorstore, FAISS.

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
loader = WebBaseLoader("https://docs.smith.langchain.com")

embeddings = OpenAIEmbeddings()
docs = loader.load()

In [10]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could pass the documents directly like previous example:

```python
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})
```

But we want to use retrieve the most relevant info from the vectorstore we just created.

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing the capability to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It also seamlessly integrates with LangChain, which is an open source framework for building with LLMs. Additionally, LangSmith offers tracing and evaluation capabilities to assist with testing LLM applications.


In [14]:
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith support